In [ ]:
# user defined R installation
import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #path to your R installation
%load_ext autoreload
%autoreload 2



import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector


from causaloptim_py.causaloptim_python import run_experiment, extract_prob_dict

import matplotlib.pyplot as plt
import seaborn as sns
import math




# Load R packages
#igraph = importr('igraph')
causaloptim = importr('causaloptim')
base = importr('base')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['c:\\Python312\\python312.zip', 'c:\\Python312\\DLLs', 'c:\\Python312\\Lib', 'c:\\Python312', '', 'C:\\Users\\tmari\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\tmari\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\tmari\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\tmari\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Python312\\Lib\\site-packages', 'causaloptim_py', 'd:/TUM Workspace/Thesis Workspace/causal-bounds', 'd:/TUM Workspace/Thesis Workspace/causal-bounds', 'd:/TUM Workspace/Thesis Workspace/causal-bounds']


ModuleNotFoundError: No module named 'causaloptim_py'